In [1]:
import queue
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = 999

In [2]:
def buildTree(arr, size):   
    state = 1
    
    for i, word in enumerate(arr):
        presentState = 0

        for c in word:
            ch = ord(c) - ord('a')
            if gotoMat[presentState][ch] == -1:
                gotoMat[presentState][ch] = state
                state = state + 1
            presentState = gotoMat[presentState][ch]

        output[presentState] = output[presentState] | (1 << i)
                
    for ch in range(MAXC):
        if gotoMat[0][ch] == -1:
            gotoMat[0][ch] = 0
            
    q = queue.Queue()
    
    for ch in range(MAXC):
        if gotoMat[0][ch] != 0:
            fail[gotoMat[0][ch]] = 0
            q.put(gotoMat[0][ch])

    while (q.qsize()):
        state = q.get()
        
        for ch in range(MAXC):
            if gotoMat[state][ch] != -1:
                failure = fail[state]
                while gotoMat[failure][ch] == -1:
                    failure = fail[failure]
                failure = gotoMat[failure][ch]
                fail[gotoMat[state][ch]] = failure
                output[gotoMat[state][ch]] = output[gotoMat[state][ch]] | output[failure]
                q.put(gotoMat[state][ch])
    return state

In [3]:
def getNextState(presentState, nextChar):
    answer = presentState
    ch = ord(nextChar) - ord('a')
    
    while gotoMat[answer][ch] == -1:
        answer = fail[answer]
    
    return gotoMat[answer][ch]

In [4]:
def patternSearch(arr, size, text):
    buildTree(arr, size)
    presentState = 0
    out = []
    
    for i, t in enumerate(text):
        presentState = getNextState(presentState, t)
        out.append(presentState)
#         if output[presentState] != 0:
#             for j in range(size+1):
#                 if output[presentState] & (1 << j):
#                     print('1Word %s appears from %s to %s' % (str(arr[j]), str(i - len(arr[j]) + 1), str(i)))
                    
    return out

In [5]:
def getPi(pattern):
    pi = [0]
    for i in range(int(len(pattern)) - 1):
        subp = pattern[:i+2]
        l = round(len(subp)/2-0.499)
        while subp[:l] != subp[(l*-1):] or l == 0:
            l -= 1
        pi.append(l if l >= 0 else 0)
    
    return pi

def kmp(text, pattern):
    pi = getPi(pattern)
    n = len(text)
    m = len(pattern)
    
    result = []
    j = 0
    for i in range(n):
        while j > 0 and text[i] != pattern[j]:
            j = pi[j-1]

        if text[i] == pattern[j]:
            if j == m-1:
                result.append(i - m + 1)
                j = pi[j]
            else:
                j += 1

    return result

In [12]:
def baker_bird(t, p):   
    ta = []
    pa = []

    # 전체 Text Matrix의 Column을 하나의 긴 Text로 취급
    # Pattern의 각 Column을 하나의 Pattern으로 취급
    # Text에서 여러 Pattern을 뽑는 Aho Corasick Algorithm 이용
    # Text(Text의 각 Column)의 각 Character의 위치(Out)를 배열로 Return
    for ti in list(map(list, zip(*t))):
        output = [0] * MAXS
        fail = [-1] * MAXS
        gotoMat = [[-1]*MAXC for _ in range(MAXS)]
        
        arr = list(reversed([''.join(pi) for pi in list(map(list, zip(*p)))]))
        arr = ['aab', 'abc', 'cba']
        text = ''.join(ti)
        k = len(arr)
        ta.append(patternSearch(arr, k, text))

    # Pattern의 각 Column을 하나의 긴 Text로 취급
    # Pattern의 각 Column을 하나의 Pattern으로 취급
    # Text에서 여러 Pattern을 뽑는 Aho Corasick Algorithm 이용
    # Pattern(Pattern의 각 Column)의 위치(Out)을 배열로 Return
    for pi in list(map(list, zip(*p))):
        output = [0] * MAXS
        fail = [-1] * MAXS
        gotoMat = [[-1]*MAXC for _ in range(MAXS)]

        arr = list(reversed([''.join(pi) for pi in list(map(list, zip(*p)))]))
        arr = ['aab', 'abc', 'cba']
        text = ''.join(pi)
        k = len(arr)
        pa.append(patternSearch(arr, k, text))
        
    # Aho Corasick Algorithm으로 변환된 Out을 Transpose 해서 원래 형태로 변환한 뒤 각 Row을 Iterate
    for r, ti in enumerate(list(map(list, zip(*ta)))):
        # 각 Row의 Out값을 하나의 Text로 취급
        text = ''.join([str(t) for t in ti])

        # Pattern의 Out값 중 마지막(즉, 전체 Pattern이 맞았을 때의 Out)값들의 배열을 Pattern으로 취급
        pattern = ''.join([str(p) for p in [o[-1] for o in pa]])
        
        # KMP Algorithm을 이용하여 각 Row에서 원하는 Pattern을 찾음
        # 즉, Text Column의 Elements 순서가 Pattern Column의 Elements 순서와 같은 놈의 마지막 Row를 찾는 형태
        sc = kmp(text, pattern)[0] if len(kmp(text, pattern)) > 0 else -1

        # KMP Algorithm으로 찾아진 경우
        # Pattern의 마지막 Row는 현재 Iterator의 Index: for문의 r
        # Pattern의 첫 Column은 KMP Algorithm의 Return 값: sc
        # n by m 행렬 패턴과 일치하는 위치는 (r - Pattern Matrix Row의 길이 + 1, sc) 부터 (r, sc + Pattern Matrix Column 의 길이 - 1)
        if sc >= 0:
            print('This pattern is from (%s, %s) to (%s, %s)' % (str(r - len(p) + 1), str(sc), str(r), str(sc + len(pattern) - 1)))
            display(pd.DataFrame(t).iloc[r-len(p)+1:r+1, sc:sc+len(pattern)])

In [19]:
MAXS = 500
MAXC = 26

output = [0] * MAXS
fail = [-1] * MAXS
gotoMat = [[-1]*MAXC for _ in range(MAXS)]

t = [['b', 'b', 'a', 'b', 'b', 'a', 'b'],
['a', 'a', 'c', 'a', 'c', 'b', 'a'],
['b', 'b', 'b', 'a', 'c', 'a', 'c'],
['a', 'c', 'a', 'b', 'b', 'a', 'b'],
['c', 'a', 'a', 'c', 'a', 'b', 'a'],
['b', 'b', 'b', 'b', 'a', 'c', 'c'],
['a', 'c', 'c', 'a', 'b', 'a', 'b']]

p = [['a', 'c', 'a', 'c'],
['b', 'b', 'a', 'b'],
['c', 'a', 'b', 'a']]

print('Pattern is')
display(pd.DataFrame(p))

baker_bird(t, p)
print('--------------------------------------------')

p = [['a', 'c', 'a'],
['b', 'b', 'a'],
['c', 'a', 'b'],
['b', 'a', 'c']]

print('Pattern is')
display(pd.DataFrame(p))

baker_bird(t, p)
print('--------------------------------------------')

p = [['a', 'c', 'a'],
['b', 'b', 'a'],
['c', 'a', 'b']]

print('Pattern is')
display(pd.DataFrame(p))

baker_bird(t, p)

Pattern is


,0,1,2,3
0,a,c,a,c
1,b,b,a,b
2,c,a,b,a


This pattern is from (2, 3) to (4, 6)


,3,4,5,6
2,a,c,a,c
3,b,b,a,b
4,c,a,b,a


--------------------------------------------
Pattern is


...

,1,2,3
1,a,c,a
2,b,b,a
3,c,a,b


This pattern is from (2, 3) to (4, 5)


,3,4,5
2,a,c,a
3,b,b,a
4,c,a,b


This pattern is from (4, 2) to (6, 4)


,2,3,4
4,a,c,a
5,b,b,a
6,c,a,b
